In [1]:
import pandas as pd
import sys
sys.path.append("../utils")

from utils import load_csv, save_data

In [2]:
df = load_csv("../../data/raw/fpt.csv")
df.head()

2025-05-27 18:58:52,309 - INFO - ✅ Load CSV thành công: ../../data/raw/fpt.csv | shape = (4553, 6)


,date,open,high,low,close,volume
0,NaN,NaN,NaN,NaN,NaN,NaN
1,27/05/2025,"117,400","119,400","117,400","119,000","5,498,200"
2,26/05/2025,"116,700","117,800","115,000","117,300","4,023,400"
3,23/05/2025,"116,700","117,800","116,000","117,100","4,440,500"
4,22/05/2025,"117,100","117,600","115,600","116,800","6,350,700"


In [3]:
df.dtypes

date      object
open      object
high      object
low       object
close     object
volume    object
dtype: object

In [36]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    original_rows = len(df)

    # Xoá các dòng toàn bộ là NaN hoặc chuỗi rỗng
    df = df.dropna(how='all')
    df = df[~(df.apply(lambda row: row.astype(str).str.strip().eq('').all(), axis=1))]
    
    # Chuẩn hóa tên cột
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

    # Xoá trùng lặp
    df = df.drop_duplicates()

    # Chuẩn hóa kiểu dữ liệu
    df['date'] = pd.to_datetime(df['date'], format="%d/%m/%Y")

    for col in ['open', 'high', 'low', 'close']:
        df[col] = df[col].str.replace(",", "").astype(float)
        df[col] = (df[col] / 1000).round(2)
    
    df['volume'] = df['volume'].str.replace(",", "").astype(int)


    # Điền missing
    for col in df.columns:
        if df[col].dtype in [int, float, float]:
            df[col] = df[col].fillna(df[col].mean())
        else:
            df[col] = df[col].fillna(df[col].mode().iloc[0])

    return df


In [38]:
# Tạo danh sách mã cổ phiếu cần xử lý
symbols = ["fpt", "hpg", "vnm"]
dfs = {}

# Lặp qua từng mã, load và tiền xử lý
for symbol in symbols:
    raw_path = f"../../data/raw/{symbol}.csv"
    df = load_csv(raw_path)
    df_clean = preprocess(df)
    dfs[symbol] = df_clean
    print(f"✅ {symbol.upper()} - Rows: {df_clean.shape[0]}, Columns: {df_clean.shape[1]}")


2025-05-27 19:29:04,783 - INFO - ✅ Load CSV thành công: ../../data/raw/fpt.csv | shape = (4552, 6)


2025-05-27 19:29:05,872 - INFO - ✅ Load CSV thành công: ../../data/raw/hpg.csv | shape = (4349, 6)


✅ FPT - Rows: 4551, Columns: 6


2025-05-27 19:29:06,900 - INFO - ✅ Load CSV thành công: ../../data/raw/vnm.csv | shape = (4784, 6)


✅ HPG - Rows: 4348, Columns: 6
✅ VNM - Rows: 4783, Columns: 6


In [39]:
# Hiển thị 5 dòng đầu tiên của mỗi mã cổ phiếu
for symbol in symbols:
    print(f"\n{symbol.upper()} - Dữ liệu sau xử lý:")
    display(dfs[symbol].head())


FPT - Dữ liệu sau xử lý:


,date,open,high,low,close,volume
1,2025-05-27,117.4,119.4,117.4,119.0,5498200
2,2025-05-26,116.7,117.8,115.0,117.3,4023400
3,2025-05-23,116.7,117.8,116.0,117.1,4440500
4,2025-05-22,117.1,117.6,115.6,116.8,6350700
5,2025-05-21,120.0,120.0,117.0,118.3,6024400



HPG - Dữ liệu sau xử lý:


,date,open,high,low,close,volume
1,2025-05-27,25.55,26.15,25.55,25.55,26892800
2,2025-05-26,25.65,25.65,25.05,25.55,27433800
3,2025-05-23,25.75,25.85,25.55,25.60,11096000
4,2025-05-22,25.75,25.95,25.55,25.65,24363700
5,2025-05-21,25.75,26.10,25.65,25.75,20150500



VNM - Dữ liệu sau xử lý:


,date,open,high,low,close,volume
1,2025-05-27,55.9,56.0,55.3,55.4,3680800
2,2025-05-26,55.4,55.8,55.0,55.7,2729200
3,2025-05-23,55.2,55.5,55.1,55.3,1666700
4,2025-05-22,55.4,55.6,55.0,55.0,2665200
5,2025-05-21,56.0,56.0,55.4,55.4,4464400


In [40]:
# Save
for symbol in symbols:
    path = f"../../data/clean/{symbol}.csv"
    save_data(dfs[symbol], f"../../data/clean/{symbol}.csv")

2025-05-27 19:29:13,775 - INFO - 💾 Đã lưu dữ liệu thành công: ../../data/clean/fpt.csv


2025-05-27 19:29:13,795 - INFO - 💾 Đã lưu dữ liệu thành công: ../../data/clean/hpg.csv
2025-05-27 19:29:13,816 - INFO - 💾 Đã lưu dữ liệu thành công: ../../data/clean/vnm.csv


In [ ]:
%pip install pymongo
from pymongo import MongoClient, errors

In [42]:
# Kết nối MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client['vn_project']

for symbol in symbols:
    collection = db[symbol]

    # Đảm bảo 'date' là unique (chỉnh lại nếu bạn dùng cột khác làm unique)
    collection.create_index('date', unique=True)

    records = dfs[symbol].to_dict(orient='records')
    
    success = 0
    for record in records:
        try:
            collection.insert_one(record)
            success += 1
        except errors.DuplicateKeyError:
            continue  # Bỏ qua bản ghi đã tồn tại

    print(f"Symbol {symbol.upper()}: đã thêm {success} bản ghi mới.")


Symbol FPT: đã thêm 4551 bản ghi mới.
Symbol HPG: đã thêm 4348 bản ghi mới.
Symbol VNM: đã thêm 4783 bản ghi mới.
